In [4]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv()

True

In [5]:
con = create_engine(f"{os.environ['DATABASE']}://{os.environ['DB_USER']}:{os.environ['DB_PASSWORD']}@{os.environ['HOST']}:{os.environ['PORT']}/{os.environ['DB_NAME']}")

In [6]:
tokenized_texts = pd.read_sql_table('tokenized_texts', con)
sentences = pd.read_sql_table('sentences', con)

In [ ]:
marker_sentences = tokenized_texts.groupby(['text_id', 'sentence_id', 'task_id', 'text_year']) \
                .filter(lambda x: x['lemma'].isin(['never', 'ever', 'always']).any())

In [ ]:
MARKERS = [r'\bnever\b', r'\bever\b', r'\balways\b', r'\bjust\b', r'\bby now\b', r'\bnowadays\b', r'\btoday\b', r'\bin all my life\b', 
           r'\bmy whole life\b', r'\blately\b', r'\brecently\b', r'\boften\b', r'\bbefore now\b', 
           r'\bat present\b', r'\bup till now\b', r'\bso far\b', r'\byet\b', r'\balready\b', 
           r'(?:\bduring\b|\bin\b|\bover the\b|\bthese past\b|\blast\b|\brecent\b)\s\d', r'\bsince\b', r'\bfor many\s\d']

marker_sentences = sentences[sentences['sentence_tokens'].str.contains('|'.join(MARKERS))]
marker_sentences['splitted_spacy_poses'] = marker_sentences['sentence_token_spacy_poses'].str.split(' ')
marker_sentences['splitted_spacy_tags'] = marker_sentences['sentence_token_spacy_tags'].str.split(' ')
marker_sentences['splitted_lemmas'] = marker_sentences['sentence_lemmas'].str.split(' ')
marker_sentences['splitted_poses'] = marker_sentences['sentence_poses'].str.split(' ')

In [ ]:
def match_pp(x, window=5):
    if 'have' in x['splitted_lemmas']:
        if x['splitted_spacy_tags'] is not None and 'VBN' in x['splitted_spacy_tags']:
            vbn_idx = x['splitted_spacy_tags'].index('VBN')
        elif 'VBN' in x['splitted_poses']:
            vbn_idx = x['splitted_poses'].index('VBN')
        else:
            return False
        return 'have' in x['splitted_lemmas'][max(0, vbn_idx-window):vbn_idx]
    return False

marker_sentences['has_pp'] = marker_sentences.apply(match_pp, axis=1)

In [169]:
marker_sentences[['sentence_tokens', 'has_pp']].to_excel('marker_sentences.xlsx', index=False)